In [1]:
from gurobipy import GRB
import gurobipy as gb

In [2]:
# Maximum difference between inventory positions of the warehouses 
max_diff = 1200

In [3]:
# The supply nodes (i.e., the warehouses)
W = 2

In [4]:
# Creates a list that has the number of units of supply for each supply node
supply = [7000, 8000]

In [5]:
# The demand nodes
B = 7

In [6]:
# Creates a list for the number of units of demand for each demand node
demand = [1000, 1800, 3600, 400, 1400, 2500, 2000]

In [7]:
# Creates a list of lists associated with the costs of each transportation path.
# From warehouse i = {A,B} to bar j = {1,2,3,4,5,6,7}. 
costs = [
         #Bars: 1 2 3 4 5 6 7
         [2.00,4.00,5.00,2.00,1.00,2.50,1.90],#A   Warehouses
         [3.00,1.00,3.00,2.00,3.00,1.75,1.60] #B
        ]

In [8]:
# Instantiate our optimization problem in
model = gb.Model("Linearize Absolute Value Constraint")

Restricted license - for non-production use only - expires 2025-11-24


In [9]:
#Construct decision variables for each class of decision variables
x = model.addVars(W, B, lb = 0, vtype=GRB.INTEGER, name="Transportation")

In [10]:
# Add the objective function to the optimization problem 
model.setObjective(gb.quicksum(x[w,b]*costs[w][b] for w in range(W) for b in range(B)), GRB.MINIMIZE)

In [11]:
# The demand minimum constraints are added to the milp variable for each demand node (bar)
model.addConstrs(gb.quicksum(x[w,b] for w in range(W)) == demand[b] for b in range(B))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

In [12]:
# The supply maximum constraints are added to the milp variable for each supply node (warehouse)
model.addConstrs(gb.quicksum(x[w,b] for b in range(B)) <= supply[w] for w in range(W))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [13]:
# The absolute value of the difference in the inventory supplied to the bars
# model.addConstr(abs(gb.quicksum(x[0,b] for b in range(B)) - gb.quicksum(x[1,b] for b in range(B)))  <= max_diff)
model.addConstr(gb.quicksum(x[0,b] for b in range(B)) - gb.quicksum(x[1,b] for b in range(B))  <= max_diff)
model.addConstr(gb.quicksum(x[1,b] for b in range(B)) - gb.quicksum(x[0,b] for b in range(B))  <= max_diff)

<gurobi.Constr *Awaiting Model Update*>

In [14]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 11 rows, 14 columns and 56 nonzeros
Model fingerprint: 0x9f5170e3
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+03]
Found heuristic solution: objective 37975.000000
Presolve removed 9 rows and 7 columns
Presolve time: 0.00s
Presolved: 2 rows, 7 columns, 14 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)
Found heuristic solution: objective 34218.000000

Root relaxation: objective 2.568750e+04, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [15]:
# Each of the variables is printed with it's resolved optimum value
total_supply = [0,0]
for v in model.getVars():    
    if ("[0," in v.varName):
        total_supply[0] += v.x
    else: 
        total_supply[1] += v.x

In [16]:
# The optimized objective function value is printed to the screen    
print("Total Cost of Transportation = ", model.objVal)
print("Supply from Warehouse A = ", total_supply[0])
print("Supply from Warehouse B = ", total_supply[1])
print("Supply Difference = ", abs(total_supply[0]-total_supply[1]))

Total Cost of Transportation =  25687.5
Supply from Warehouse A =  5750.0
Supply from Warehouse B =  6950.0
Supply Difference =  1200.0
